In [1]:
import torch as pt
import torchaudio as ta
from torch.utils.data import Dataset,DataLoader

import numpy as numpy

import os
import tqdm as tqdm
import json

F:\anacondapython\envs\tf\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [21]:
path='data\public_dataset'
for file in os.listdir(path):
    if file.endswith('webm'):
        name,ext=os.path.splitext(file)
        new_name=name+'.wav'
        os.rename(os.path.join(path,file),os.path.join(path,new_name))

In [4]:
path='data\public_dataset'
name_set=set()
for file in os.listdir(path):
    if file.endswith('webm'):
        name_set.add(file)
print(len(name_set))

25985


In [5]:
t=os.path.join(path,list(name_set)[1])

In [9]:
t=t.replace('webm','wav')

In [2]:
print(str(ta.get_audio_backend()))

soundfile


In [10]:
signal,sr=ta.load(t)

RuntimeError: Error opening 'data\\public_dataset\\01b83641-dc8e-4231-a50e-32ede1ffd5e2.wav': System error.

In [13]:
t=t.replace('webm','json')
with open(t,'r') as f:
    content=json.loads(f.read())
print(content)

{'datetime': '2020-04-11T13:35:31.037321+00:00', 'cough_detected': '0.9902', 'latitude': '37.4', 'longitude': '-6.1'}


In [16]:
print(content['cough_detected'])

0.9902


In [ ]:
class CoughVidDataset(Dataset):

    def __init__(self,path):
        name_set=set()
        for file in os.listdir(path):
            if file.endswith('webm'):
                name_set.add(file)
        name_set=list(name_set)
        self.datalist=name_set
        self.path=path
        
    def __len__(self):
        return len(self.datalist)

    def __getitem__(self,idx):
        audio_path=os.path.join(self.path,self.datalist[idx])
